In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import defaultdict
# from tqdm.notebook import tqdm
from  sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)
import gc

In [57]:
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val
        # functions / callables
        if hasattr(val, '__call__'):
            yield name, val

noglobal = lambda fn: types.FunctionType(fn.__code__, dict(imports()))

In [59]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

a = 1

@noglobal
def f(b):
#     h5py.is_hdf5("a.tmp")
    # only np. shall be known, not numpy.
#     np.arange(4)
    #numpy.arange(4)
    # this var access shall break when called
#     print(a)
    print(a+b)

In [60]:
f(2)

NameError: name 'a' is not defined

In [2]:
def _reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
def add_user_feats(df, answered_correctly_sum_u_dict, count_u_dict):
    acsu = np.zeros(len(df), dtype=np.int32)
    cu = np.zeros(len(df), dtype=np.int32)
    for cnt,row in enumerate(tqdm(df[['user_id','answered_correctly']].values)):
        acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
        cu[cnt] = count_u_dict[row[0]]
        answered_correctly_sum_u_dict[row[0]] += row[1]
        count_u_dict[row[0]] += 1
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    return df

In [11]:
train = pd.read_feather('../features/mini_data/BASE_train.feather')
valid = pd.read_feather('../features/mini_data/BASE_valid.feather')

In [14]:
answered_correctly_sum_u_dict = defaultdict(int)
count_u_dict = defaultdict(int)
train = add_user_feats(train, answered_correctly_sum_u_dict, count_u_dict)

100%|██████████| 8684585/8684585 [00:13<00:00, 665525.20it/s]


In [16]:
for cnt,row in enumerate(tqdm(valid[['user_id','answered_correctly']].values)):
#     acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
    cnt
    row

100%|██████████| 2485495/2485495 [00:00<00:00, 2990517.94it/s]


In [24]:
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tags1,tags2,tags3,tags4,tags5,tags6,tag,type_of,l_type_of,answered_correctly_sum_u,count_u,answered_correctly_avg_u
0,8511,0,44331,5542,0,0,1,0,NaN,NaN,5542.0,3.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0,0,NaN
1,8512,35028,44331,5697,0,1,2,0,44000.0,0.0,5697.0,0.0,5,14,14.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0,1,0.000000
2,8513,61706,44331,5748,0,2,3,1,32000.0,0.0,5748.0,3.0,5,8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0,2,0.000000
3,8514,85685,44331,376,0,3,0,0,24000.0,0.0,376.0,1.0,2,62 90 100 38 102,62.0,90.0,100.0,38.0,102.0,NaN,NaN,None,NaN,1,3,0.333333
4,8515,141228,44331,5597,0,4,3,1,21000.0,0.0,5597.0,3.0,5,64,64.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,1,4,0.250000


In [29]:
# count_u_dict

In [25]:
len(answered_correctly_sum_u_dict)

52355

In [27]:
train['user_id'].nunique()

52355

In [35]:
answered_correctly_sum_u_dict = defaultdict(int)

In [36]:
answered_correctly_sum_u_dict[115] += 1

In [39]:
answered_correctly_sum_u_dict[115]

1

In [43]:
train['user_id_part'] = train['user_id'].astype(str) + '-' +train['part'].astype(str)

In [44]:
train

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tags1,tags2,tags3,tags4,tags5,tags6,tag,type_of,l_type_of,answered_correctly_sum_u,count_u,answered_correctly_avg_u,user_id_part
0,8511,0,44331,5542,0,0,1,0,NaN,NaN,5542.0,3.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0,0,NaN,44331-5
1,8512,35028,44331,5697,0,1,2,0,44000.0,0.0,5697.0,0.0,5,14,14.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0,1,0.000000,44331-5
2,8513,61706,44331,5748,0,2,3,1,32000.0,0.0,5748.0,3.0,5,8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0,2,0.000000,44331-5
3,8514,85685,44331,376,0,3,0,0,24000.0,0.0,376.0,1.0,2,62 90 100 38 102,62.0,90.0,100.0,38.0,102.0,NaN,NaN,None,NaN,1,3,0.333333,44331-2
4,8515,141228,44331,5597,0,4,3,1,21000.0,0.0,5597.0,3.0,5,64,64.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,1,4,0.250000,44331-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8684580,101228658,219919,2147463192,6318,0,12,3,0,9000.0,1.0,6318.0,0.0,5,124,124.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,9,14,0.642857,2147463192-5
8684581,101228659,233275,2147463192,4756,0,13,0,0,6000.0,1.0,4756.0,2.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,9,15,0.600000,2147463192-5
8684582,101228660,253051,2147463192,9376,0,14,0,0,5000.0,1.0,9376.0,2.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,9,16,0.562500,2147463192-5
8684583,101228661,272506,2147463192,4447,0,15,1,1,10000.0,1.0,4447.0,1.0,5,47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,9,17,0.529412,2147463192-5


In [62]:
features_list = ['BASE','USER_ID','PART','USER_PART','CONTENT','USER_CONTENT','TAGS']

dfs = [pd.read_feather(f'../features/mini_data/{feature}_train.feather') for feature in features_list]
df = pd.concat(dfs,axis=1)

In [64]:
df

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tags1,tags2,tags3,tags4,tags5,tags6,tag,type_of,l_type_of,answered_correctly_avg_user,answered_correctly_sum_user,count_user,answered_correctly_avg_part,answered_correctly_avg_user_part,answered_correctly_sum_user_part,count_user_part,answered_correctly_avg_content,answered_correctly_sum_content,content_num,answered_correctly_avg_user_content,answered_correctly_sum_user_content,count_user_content,answered_correctly_avg_tags1
0,8511,0,44331,5542,0,0,1,0,NaN,NaN,5542.0,3.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0,0,0.602282,NaN,0,0,0.752778,271,360,NaN,0,0,0.607741
1,8512,35028,44331,5697,0,1,2,0,44000.0,0.0,5697.0,0.0,5,14,14.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,0,1,0.602282,0.000000,0,1,0.579035,348,601,NaN,0,0,0.695563
2,8513,61706,44331,5748,0,2,3,1,32000.0,0.0,5748.0,3.0,5,8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,0,2,0.602282,0.000000,0,2,0.623457,303,486,NaN,0,0,0.610308
3,8514,85685,44331,376,0,3,0,0,24000.0,0.0,376.0,1.0,2,62 90 100 38 102,62.0,90.0,100.0,38.0,102.0,NaN,NaN,None,NaN,0.333333,1,3,0.701379,NaN,0,0,0.670373,1869,2788,NaN,0,0,0.735485
4,8515,141228,44331,5597,0,4,3,1,21000.0,0.0,5597.0,3.0,5,64,64.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.250000,1,4,0.602282,0.333333,1,3,0.828957,1168,1409,NaN,0,0,0.610358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8684580,101228658,219919,2147463192,6318,0,12,3,0,9000.0,1.0,6318.0,0.0,5,124,124.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.642857,9,14,0.602282,0.666667,8,12,0.598711,4458,7446,NaN,0,0,0.623604
8684581,101228659,233275,2147463192,4756,0,13,0,0,6000.0,1.0,4756.0,2.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.600000,9,15,0.602282,0.615385,8,13,0.722628,396,548,NaN,0,0,0.607741
8684582,101228660,253051,2147463192,9376,0,14,0,0,5000.0,1.0,9376.0,2.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.562500,9,16,0.602282,0.571429,8,14,0.860606,284,330,NaN,0,0,0.607741
8684583,101228661,272506,2147463192,4447,0,15,1,1,10000.0,1.0,4447.0,1.0,5,47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.529412,9,17,0.602282,0.533333,8,15,0.707136,2091,2957,NaN,0,0,0.551334


In [65]:
df['answered_correctly_avg_user_content'].isnull().sum()

7692882

In [73]:
df['dd'] = 0
df.loc[df['user_id'] == 44331,'dd'] = 1

In [77]:
df_part_train = pd.read_feather(f'../features/mini_data/PART_train.feather')
df_part_valid = pd.read_feather(f'../features/mini_data/PART_valid.feather')

In [80]:
df_part_train['reading_part'].isnull().sum()

0

In [81]:
df_part_valid['reading_part'].isnull().sum()

0

In [3]:
train = pd.read_feather(f'../features/mini_data/BASE_train.feather')
valid = pd.read_feather(f'../features/mini_data/BASE_valid.feather')

In [87]:
train = pd.concat([train,df_part_train],axis=1)
valid = pd.concat([valid,df_part_train],axis=1)

In [89]:
train['reading_part'].isnull().sum()

0

In [90]:
valid['reading_part'].isnull().sum()

0

In [4]:
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tags1,tags2,tags3,tags4,tags5,tags6,tag,type_of,l_type_of
0,8511,0,44331,5542,0,0,1,0,NaN,NaN,5542.0,3.0,5,96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,8512,35028,44331,5697,0,1,2,0,44000.0,0.0,5697.0,0.0,5,14,14.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,8513,61706,44331,5748,0,2,3,1,32000.0,0.0,5748.0,3.0,5,8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,8514,85685,44331,376,0,3,0,0,24000.0,0.0,376.0,1.0,2,62 90 100 38 102,62.0,90.0,100.0,38.0,102.0,NaN,NaN,None,NaN
4,8515,141228,44331,5597,0,4,3,1,21000.0,0.0,5597.0,3.0,5,64,64.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [11]:
train['']
train[['type_of','part']].groupby('type_of').agg(['mean'])

,part
,mean
type_of,
concept,4.897378
intention,2.000000
solving question,3.748099
starter,5.000000


In [13]:
train['type_of'].fillna('NAN')

0          NAN
1          NAN
2          NAN
3          NAN
4          NAN
          ... 
8684580    NAN
8684581    NAN
8684582    NAN
8684583    NAN
8684584    NAN
Name: type_of, Length: 8684585, dtype: object

In [9]:
8568610/train.shape[0]

0.9866458788761927